# Awesome NTDS project 

bla bla bla

## 1. Dependencies

*Matplotlib* do not cope well with large scale 3D data visualization. This can be overcomed by the use of `mayavi` package (*build on top of OpenGL pipeline*). It can be installed on your machine with the following commands:

```
$ conda install -c clinicalgraphics vtk=7.1.0
$ pip install mayavi
```

In order to use it in a `jupyter` notebook, some extension need to be installed. The command listed below will do it:

```
$ jupyter nbextension install --py mayavi --user
```


In [1]:
from utils import *
from mesh import *
from deformation import *
import numpy as np
import os
from scipy import sparse
#from mayavi import mlab
#mlab.init_notebook(local=False)

# Base folder where data are located, user-specific since location
data_folder = '../data'

## 2. Data Analysis

Can we recover some morphological properties such as facial expressions, or ethnies within the dataset ?

## 3. Graph-based deformation learning

### 3.1 Deformation field

In this section, the learning of a deformation field $\boldsymbol{d}$ using graph-based approach will be investigated in the following section.

It can be formulated as follow; given a source surface $\boldsymbol{x}_s$ can we estimate a deformation field $\boldsymbol{d}$ using only a sparse subset of the target surface $\boldsymbol{x}_t$.

In [2]:
# Load source + target meshes + anchors
tri = utils.load_triangulation(os.path.join(data_folder, 'FWTri/fw_triangulation.tri'))
meshes = utils.load_meshes(os.path.join(data_folder, 'FWMesh/115_shape.bs'), [0, 22])
anchors = utils.load_anchor_point(os.path.join(data_folder, 'anchors.cfg'))

In [13]:
# Create mesh instances for source and target surface
mesh_xs = Mesh(meshes[0], tri)
mesh_xt = Mesh(meshes[1], tri)
N = meshes[0].shape[0]

In [ ]:
mlab.clf()
fig = mlab.figure()
color = (0.5, 0.5, 0.5)
trimesh_xs = mlab.triangular_mesh(mesh_xs.vertex[:,0], mesh_xs.vertex[:,1], mesh_xs.vertex[:,2], mesh_xs.tri, color=color)
mlab.view(0,0)
fig

In [ ]:
mlab.clf()
fig = mlab.figure()
trimesh_xt = mlab.triangular_mesh(mesh_xt.vertex[:,0], mesh_xt.vertex[:,1], mesh_xs.vertex[:,2], mesh_xt.tri, color=color)
mlab.view(0, 0)
fig

In [4]:
# 1) create selection mask
K = len(anchors)
ridx = [k for k in range(K)]
cidx = anchors
data = [1.0] * K
M = sparse.coo_matrix((data, (ridx, cidx)), shape=(K, N), dtype=np.float32)
# 2) Comptue laplacian
_,_, Lap = mesh_xs.compute_laplacian('cotan')
# 3) Compute target
Xs = mesh_xs.vertex
Xt = M.dot(mesh_xt.vertex)

In [5]:
# 4) Estimate deformation field without regularization
estm_xt, d = deform_regularized_anchor(Xs, Xt, M, Lap, 1.0)
err = np.linalg.norm(estm_xt - mesh_xt.vertex, axis=1)
e = np.mean(err)
print('Mean error of %f', e)
m = Mesh(estm_xt, tri)
m.save('deformation_field.obj')

Mean error of %f 0.00449361


In [ ]:
mlab.clf()
fig = mlab.figure()
trimesh_xt_estm = mlab.triangular_mesh(estm_xt[:,0], estm_xt[:,1], estm_xt[:,2], mesh_xt.tri, color=color)
mlab.view(0, 0)
fig

### 3.2 Least-Square Mesh

bla bla

In [14]:
# Define only anchors around the lips
anchorsIdx = [703, 4293, 7187, 2128, # Eyebrows
              9397, 8972, # Nose
              10308, 3237, 3205, 6081, 6091,6124,8832, 6611, 1827, 3582, 402, 3584] # Mouth region
# Inner part
# 3271, 10313, 8808, 8825, 6159, 8842, 8809, 10326
anchors = mesh_xt.vertex[anchorsIdx, :]
# Deform
mesh = deform_mesh(mesh_xs, anchors, anchorsIdx, 1.0)
estm_xt = mesh.vertex

In [15]:
# Error
err = np.linalg.norm(estm_xt - mesh_xt.vertex, axis=1)
e = np.mean(err)

print('Mean error: %f' % e)

Mean error: 0.246112


In [16]:
mesh.save('least_square_mesh.obj')